In [1]:
#!pip install transformers
import json
import pandas as pd
import numpy as np
#import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from collections import defaultdict as ddict
from transformers import *
from tqdm import tqdm, trange

from bertviz.bertviz import head_view

In [2]:
# 02/2021: Add verb to beginning of sentence for input
# Note: add_verb doesn't work because the back-translated data don't have verbs marked (yet)

add_verb = False

#augment_data = 'none'
augment_data= 'translated'
#augment_data = 'r2r'

In [3]:
print("GPU Available: {}".format(torch.cuda.is_available()))
n_gpu = torch.cuda.device_count()
print("Number of GPU Available: {}".format(n_gpu))
print("GPU: {}".format(torch.cuda.get_device_name(0)))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU Available: True
Number of GPU Available: 4
GPU: Tesla V100-SXM2-32GB


In [4]:
#df = pd.read_pickle("../data/fd_vn_physical_with_tokens_and_pooled_output_v4.pkl")

#df = pd.read_pickle("../data/fd_vn_physical_with_augmented_v6.pkl")

df = pd.read_pickle("../data/fd_vn_physical_with_augmented_r2r_v7.pkl")

In [5]:
# if add_verb:
#     df['sentence'] = df['verb'] + ' ' + df['sentence']

df[df['augmented']==1]

,pk,cc,sentence,syntax,fd1,fd2,fd3,verb,augmented,dep
565,664,"[{'text': 'cart', 'q_value': 'INTL', 'index': ...",Grocery frame hit wall,NaN,Physical,Force,none,NaN,1,NaN
566,664,"[{'text': 'cart', 'q_value': 'INTL', 'index': ...",The branch struck the wall,NaN,Physical,Force,none,NaN,1,NaN
567,664,"[{'text': 'cart', 'q_value': 'INTL', 'index': ...",The grocery truck hit the wall,NaN,Physical,Force,none,NaN,1,NaN
568,664,"[{'text': 'cart', 'q_value': 'INTL', 'index': ...",The fuel trolley hit the wall,NaN,Physical,Force,none,NaN,1,NaN
569,664,"[{'text': 'cart', 'q_value': 'INTL', 'index': ...",The rear of the cart struck the wall,NaN,Physical,Force,none,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...
2132,433.3,NaN,Inject the fluid into the machine,NaN,Volitional,Place,none,NaN,1,NaN
2133,433.4,NaN,Pour water on seed and hold,NaN,Volitional,Place,none,NaN,1,NaN
2134,433.4,NaN,Spill the substance onto the earth,NaN,Volitional,Place,none,NaN,1,NaN
2135,433.4,NaN,Put water on flower,NaN,Volitional,Place,none,NaN,1,NaN


In [6]:
def integrate_propara_data_training():
    
    df_propara = pd.read_pickle("~/src/coling2020-code/data/propara_fd_image_schema_v4.pkl")
    
    df_propara['fd'] = df_propara.apply (lambda row: (row.fd1+" "+row.fd2+" "+row.fd3), axis=1)
     
    return df_propara

In [7]:
#df_propara = integrate_propara_data_training()


#df = pd.concat([df, df_propara], axis=0)

In [8]:
df.head()

,pk,cc,sentence,syntax,fd1,fd2,fd3,verb,augmented,dep
0,664,"[{'text': 'cart', 'q_value': 'INTL', 'index': ...",The grocery cart hit against the wall,Sbj V PathP,Physical,Force,none,hit,0,NaN
1,665,"[{'text': 'rope', 'q_value': '+MER', 'index': ...",That type of rope coiled easily around the post,Sbj V PathP,Autonomous,Place,none,coil,0,NaN
2,666,"[{'text': 'rope', 'q_value': '+MER', 'index': ...",The rope coiled around the post,Sbj V PathP,Autonomous,Place,none,coil,0,NaN
3,667,"[{'text': 'company', 'q_value': '+MER', 'index...",The company is wedging into new markets,Sbj V PathP,Autonomous,Place,none,wedge,0,NaN
4,669,"[{'text': 'computer', 'q_value': '+MER', 'inde...",The computer connected well to the network,Sbj V PathP,Autonomous,Place,none,connect,0,NaN


In [9]:
fd1_cnt = ddict(int)
fd2_cnt = ddict(int)
fd3_cnt = ddict(int)

# _fd = df.fd.unique()
all_fd1 = []
all_fd2 = []
all_fd3 = []
for idx, row in df.iterrows():
    #fd = row.fd

    #fd1 = fd.split()[0].lower()
    fd1 = row.fd1.lower()
    #fd2 = fd.split()[1].lower()
    fd2 = row.fd2.lower()
    fd3 = row.fd3.lower()

    fd1_cnt[fd1] += 1
    fd2_cnt[fd2] += 1
    fd3_cnt[fd3] += 1

    all_fd1.append(fd1)
    all_fd2.append(fd2)
    all_fd3.append(fd3)
        
#         if len(fd.split())>2:
#             fd3 = fd.split()[2].lower()
#             fd3_cnt[fd3] += 1
#             all_fd3.append(fd3)
#         else:
#             fd3_cnt['none'] += 1
all_fd1 = sorted(list(set(all_fd1)))
all_fd2 = sorted(list(set(all_fd2)))
all_fd3 = sorted(list(set(all_fd3)))
#all_fd3.append("None")
print(all_fd1)
print(all_fd2)
print(all_fd3)

['autonomous', 'cause', 'instrument', 'manipulate', 'mutual', 'physical', 'self-volitional', 'volitional']
['attend', 'cos', 'create', 'deprive', 'dynamictexture', 'force', 'form', 'internal', 'location', 'motion', 'place', 'provide', 'remove']
['affect', 'none', 'purp', 'xpr']


In [10]:
fd1_cnt

defaultdict(int,
            {'physical': 83,
             'autonomous': 550,
             'self-volitional': 199,
             'volitional': 61361,
             'manipulate': 25,
             'cause': 35,
             'instrument': 113,
             'mutual': 14})

In [11]:
fd2_cnt

defaultdict(int,
            {'force': 139,
             'place': 262,
             'internal': 486,
             'cos': 545,
             'provide': 61,
             'create': 138,
             'dynamictexture': 30,
             'form': 22,
             'attend': 10,
             'location': 13984,
             'motion': 46643,
             'remove': 56,
             'deprive': 4})

In [12]:
fd3_cnt

defaultdict(int, {'none': 62272, 'affect': 69, 'purp': 30, 'xpr': 9})

In [13]:
def get_all_items_from_df(dff, augment_data='none'):

    all_items = []
    
    if augment_data=='none':
        dff = dff[dff['augmented']==0]
    elif augment_data=='translated':
        dff = dff.loc[(dff['augmented'] == 0) | (dff['augmented'] == 1)]
        
    else:
        dff_r2r = dff[dff['augmented']==2]
        dff_sample = dff_r2r.sample(frac=0.01, replace=False, random_state=1)
        
        dff = dff.loc[(dff['augmented'] == 0) | (dff['augmented'] == 1)]
        
        dff = pd.concat([dff, dff_sample])
        
        
        

    for idx, row in dff.iterrows(): 
        sentence = row.sentence
        item = {"sentence":sentence, 
            'physical': 0, 'mutual':0, 'self-volitional':0, 'volitional':0, 'instrument':0, 'autonomous':0, 'cause':0, 'manipulate': 0,
            'internal': 0, 'force': 0, 'provide': 0, 'form': 0, 'place': 0, 'motion': 0, 'dynamictexture': 0, 
            'remove': 0, 'create': 0, 'location': 0, 'deprive': 0, 'cos': 0, 'attend':0,
            'xpr': 0, 'purp': 0, 'affect': 0, 'none': 0}

        item[row.fd1.lower()] = 1 
        item[row.fd2.lower()] = 1 
        item[row.fd3.lower()] = 1 
        all_items.append(item)
        
    return pd.DataFrame(all_items)

df1 = get_all_items_from_df(df, augment_data=augment_data)

In [14]:
len(df1)

2137

In [15]:
df1.tail()

#df1.drop(['experience'], axis=1, inplace=True)

,sentence,physical,mutual,self-volitional,volitional,instrument,autonomous,cause,manipulate,internal,...,remove,create,location,deprive,cos,attend,xpr,purp,affect,none
2132,Inject the fluid into the machine,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2133,Pour water on seed and hold,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2134,Spill the substance onto the earth,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2135,Put water on flower,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2136,Give water to the plants,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
len(df1)

2137

In [17]:
print('average train sentence length: ', df1.sentence.str.split().str.len().mean())
print('stdev train sentence length: ', df1.sentence.str.split().str.len().std())

average train sentence length:  5.408984557791296
stdev train sentence length:  1.9056081603791868


In [18]:
df1 = df1.sample(frac=1).reset_index(drop=True) #shuffle rows

In [19]:
cols = df1.columns
label_cols = list(cols[1:])
num_labels = len(label_cols)
print('Label columns: ', label_cols)

Label columns:  ['physical', 'mutual', 'self-volitional', 'volitional', 'instrument', 'autonomous', 'cause', 'manipulate', 'internal', 'force', 'provide', 'form', 'place', 'motion', 'dynamictexture', 'remove', 'create', 'location', 'deprive', 'cos', 'attend', 'xpr', 'purp', 'affect', 'none']


In [20]:
all_fd3

['affect', 'none', 'purp', 'xpr']

In [21]:
df1['one_hot_labels'] = list(df1[label_cols].values)
df1['one_hot_labels_fd1'] = list(df1[all_fd1].values)
df1['one_hot_labels_fd2'] = list(df1[all_fd2].values)
df1['one_hot_labels_fd3'] = list(df1[all_fd3].values)
df1.head()

,sentence,physical,mutual,self-volitional,volitional,instrument,autonomous,cause,manipulate,internal,...,cos,attend,xpr,purp,affect,none,one_hot_labels,one_hot_labels_fd1,one_hot_labels_fd2,one_hot_labels_fd3
0,The jewel sparkled,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 1, 0, 0]"
1,A storm blew in the sky,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0, 1, 0, 0]"
2,He hit the mouse to death with spatula,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0]"
3,I recorded reading on a notepad with her,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 1, 0, 0]"
4,The militia launched the shot to the spot.,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 1, 0, 0]"


In [22]:
labels = list(df1.one_hot_labels.values)
sentences = list(df1.sentence.values)

In [23]:
# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts = df1.one_hot_labels.astype(str).value_counts()
print(label_counts)
one_freq = label_counts[label_counts==1].keys()
one_freq_idxs = sorted(list(df1[df1.one_hot_labels.astype(str).isin(one_freq)].index), reverse=True)
print('df label indices with only one instance: ', one_freq_idxs)

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1]    300
[0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]    199
[0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]    190
[0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1]    178
[0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]    124
[0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]    108
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1]    105
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]     88
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]     72
[0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1]     70
[0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]     64
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1]     64
[0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]     52
[0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]     48
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0]     47
[0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]     42
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1]     

In [24]:
pretrained_weights = 'roberta-large'

# max_length = 30
tokenizer = RobertaTokenizer.from_pretrained(pretrained_weights, do_lower_case=False, add_special_tokens=True)
encodings = tokenizer.batch_encode_plus(sentences,
#                                         max_length=max_length,
                                        padding=True,
                                        return_token_type_ids=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())

tokenizer outputs:  dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [25]:
input_ids = encodings['input_ids'] # tokenized and encoded sentences
token_type_ids = encodings['token_type_ids'] # token type ids
attention_masks = encodings['attention_mask'] # attention masks

In [26]:
# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_token_types = [token_type_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

In [27]:
# Use train_test_split to split our data into train and validation sets

train_inputs, validation_inputs, train_labels, validation_labels, train_token_types, validation_token_types, train_masks, validation_masks = train_test_split(input_ids, labels, token_type_ids,attention_masks,
                                                            random_state=2020, test_size=0.20, stratify = labels)

In [28]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_token_types = torch.tensor(validation_token_types)

In [29]:
# Select a batch size for training. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [30]:
config = RobertaConfig.from_pretrained(pretrained_weights, num_labels=num_labels)
config.output_hidden_states = True
config.output_attentions = True

model = RobertaForSequenceClassification.from_pretrained(pretrained_weights, 
                                                         config=config)

#model = RobertaForSequenceClassification.from_pretrained('roberta-large-mnli', num_labels=3)

#model = RobertaForSequenceClassification.from_pretrained('/nfs/research/regan/bin/roberta_models/', num_labels=num_labels)

model.cuda()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [31]:
# setting custom optimization parameters. You may implement a scheduler here as well.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [32]:
optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5,correct_bias=True)
# optimizer = AdamW(model.parameters(),lr=2e-5)  # Default optimization

# Number of training epochs (authors recommend between 2 and 4)
epochs = 16

num_warmup_steps = batch_size
num_total_steps = batch_size * epochs
#scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_total_steps)  # PyTorch scheduler

In [33]:
# Store our loss and accuracy for plotting
train_loss_set = []

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0 #running loss
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()

    # # Forward pass for multiclass classification
    #outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    #loss = outputs[0]

    # logits = outputs[1]

    # Forward pass for multilabel classification
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    loss_func = BCEWithLogitsLoss() 
    loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    # loss_func = BCELoss() 
    # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    train_loss_set.append(loss.item())    

    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    #scheduler.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

###############################################################################

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Variables to gather full output
  logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

  # Predict
  for i, batch in enumerate(validation_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

  # Flatten outputs
  pred_labels = [item for sublist in pred_labels for item in sublist]
  true_labels = [item for sublist in true_labels for item in sublist]

  # Calculate Accuracy
  threshold = 0.50
  pred_bools = [pl>threshold for pl in pred_labels]
  true_bools = [tl==1 for tl in true_labels]
  val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
  val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

  print('F1 Validation Accuracy: ', val_f1_accuracy)
  print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/16 [00:00<?, ?it/s]

Train loss: 0.34108948128090966


Epoch:   6%|▋         | 1/16 [00:09<02:16,  9.10s/it]

F1 Validation Accuracy:  58.971962616822424
Flat Validation Accuracy:  0.0
Train loss: 0.22500478641854393


Epoch:  12%|█▎        | 2/16 [00:17<02:06,  9.01s/it]

...

F1 Validation Accuracy:  96.39498432601881
Flat Validation Accuracy:  87.85046728971963
Train loss: 0.01889981536401643


Epoch:  94%|█████████▍| 15/16 [02:13<00:08,  8.90s/it]

F1 Validation Accuracy:  96.24413145539906
Flat Validation Accuracy:  87.85046728971963
Train loss: 0.017939501217807883


Epoch: 100%|██████████| 16/16 [02:22<00:00,  8.89s/it]

F1 Validation Accuracy:  96.47611589663273
Flat Validation Accuracy:  88.08411214953271


In [34]:
def test_single_example_to_examine_weights(model):

    sequence = ['I went to the store and went in.']
    sequence_encoding = tokenizer.batch_encode_plus(sequence,
#                                                     max_length=max_length,
                                                    padding=True,
                                                    return_token_type_ids=True, 
                                                    return_tensors="pt") # tokenizer's encoding method
    sequence_input_ids = sequence_encoding['input_ids']
    sequence_token_type_ids = sequence_encoding['token_type_ids']
    sequence_attention_masks = sequence_encoding['attention_mask']
    sequence_inputs = torch.tensor(sequence_input_ids).to(device)
    #sequence_labels = torch.tensor(sequence_labels)
    sequence_masks = torch.tensor(sequence_attention_masks).to(device)
    sequence_token_types = torch.tensor(sequence_token_type_ids)
    
    model.eval()
    
    # Predict
    with torch.no_grad():
        # Forward pass
        outs = model(sequence_inputs, token_type_ids=None, attention_mask=sequence_masks)
        print(len(outs))
#         print(outs[0].shape)
#         print(len(outs[1]))
#         print(outs[1][0].shape)
        #print(outs[2][1].shape)
        input_ids = sequence_input_ids.tolist()[0]
        tokens = tokenizer.convert_ids_to_tokens(input_ids)
        print(tokens)
        
        attention = outs[-1]
        print(len(attention))
#         for i in range(len(sent)):
#             print(outs[2][1][0][0][i])
#         print(torch.argmax(outs[2][0][0][0], dim=1))
#         print(torch.argmax(outs[2][1][0][0], dim=1))
#         print(torch.argmax(outs[2][2][0][0], dim=1))
        b_logit_pred = outs[0]
        pred_label = torch.sigmoid(b_logit_pred)

        b_logit_pred = b_logit_pred.detach().cpu().numpy()
        pred_label = pred_label.to('cpu').numpy()
        #b_labels = b_labels.to('cpu').numpy()

        pred_bools = [pl>0.50 for pl in pred_label[0]] #boolean output after thresholding
        prediction = ''
        for idx, pred in enumerate(pred_bools):
            if pred == True:
                prediction += ' ' + label_cols[idx]
        print(prediction)
    return attention, tokens

        
attention, tokens = test_single_example_to_examine_weights(model)

3
['<s>', 'I', 'Ġwent', 'Ġto', 'Ġthe', 'Ġstore', 'Ġand', 'Ġwent', 'Ġin', '.', '</s>']
24
 self-volitional motion none


/nfs/research/regan/.conda/envs/torch-roberta/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/nfs/research/regan/.conda/envs/torch-roberta/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/nfs/research/regan/.conda/envs/torch-roberta/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


In [35]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [36]:
head_view(attention, tokens)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
def get_propara_test_data():
    
    df_test = pd.read_pickle("~/src/coling2020-code/data/propara_fd_image_schema_v5.pkl")
    return df_test

In [38]:
def get_material_synthesis_data(json_file):
    with open(json_file, 'r') as infile:
        data = json.load(infile)
        
    return pd.DataFrame(data)

In [39]:
# careful when you test on propara that you have not included this in the training data

test_data = 'material'
test_data = 'propara'

json_file = "/nfs/research/regan/src/coling2020-code/data/material_synthesis_image_schema.json"

if test_data == 'material':
    df1 = get_material_synthesis_data(json_file)
else:
    df1 = get_propara_test_data()

# df_material = get_material_synthesis_data(json_file)
# df_propara = get_propara_test_data()

#dff_test = pd.concat([df_material, df_propara])

dff_test = df1

# if add_verb:
#     dff_test['sentence'] = dff_test['verb'] + ' ' + dff_test['sentence']
    

dff_test.head()

,sentence,syntax,fd1,fd2,fd3,verb
0,Plants die,Sbj V,autonomous,cos,none,die
1,They are buried in sediment,Sbj V LocP,autonomous,location,none,bury
2,Bacteria is buried in sediment,Sbj V LocP,autonomous,location,none,bury
3,Large amounts of sediment gradually pile on to...,Sbj V PathP,autonomous,place,none,pile
4,Pressure builds up,Sbj V,autonomous,cos,none,build_up


In [40]:
len(dff_test)

1314

In [41]:
fd1_cnt = ddict(int)
fd2_cnt = ddict(int)
fd3_cnt = ddict(int)

for idx, row in dff_test.iterrows():
    fd1_cnt[row.fd1.lower()] += 1
    fd2_cnt[row.fd2.lower()] += 1
    if len(row.fd3)<1:
        fd3_cnt['none'] += 1
    else:
        fd3_cnt[row.fd3.lower()] += 1
    

In [42]:
fd1_cnt

defaultdict(int,
            {'autonomous': 851,
             'physical': 297,
             'cause': 30,
             'volitional': 115,
             'self-volitional': 12,
             'instrument': 8,
             'mutual': 1})

In [43]:
fd2_cnt

defaultdict(int,
            {'cos': 341,
             'location': 41,
             'place': 124,
             'internal': 186,
             'form': 113,
             'motion': 260,
             'force': 36,
             'remove': 69,
             'create': 118,
             'experience': 9,
             'provide': 17})

In [44]:
fd3_cnt

defaultdict(int, {'none': 1276, 'xpr': 1, 'purp': 37})

In [45]:
test_all_items = []

for idx, row in dff_test.iterrows(): 
    sentence = row.sentence
    item = {"sentence":sentence, 
        'physical': 0, 'mutual':0, 'self-volitional':0, 'volitional':0, 'instrument':0, 'autonomous':0, 'cause':0, 'manipulate': 0,
        'internal': 0, 'force': 0, 'provide': 0, 'form': 0, 'place': 0, 'motion': 0, 'dynamictexture': 0, 
        'remove': 0, 'create': 0, 'location': 0, 'deprive': 0, 'cos': 0, 'attend':0,
        'xpr': 0, 'purp': 0, 'affect': 0, 'none': 0}

    if "Experience" in row.fd2:
        pass
    else:
        item[row.fd1.lower()] = 1
        item[row.fd2.lower()] = 1
        if row.fd3.lower() == 'none' or row.fd3 == '':
            item['none'] = 1
        else:
            item[row.fd3.lower()] = 1

        test_all_items.append(item)

test_df = pd.DataFrame(test_all_items)

In [46]:
test_df.head()

,sentence,physical,mutual,self-volitional,volitional,instrument,autonomous,cause,manipulate,internal,...,create,location,deprive,cos,attend,xpr,purp,affect,none,experience
0,Plants die,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,NaN
1,They are buried in sediment,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,NaN
2,Bacteria is buried in sediment,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,NaN
3,Large amounts of sediment gradually pile on to...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
4,Pressure builds up,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,NaN


In [47]:
test_label_cols = list(test_df.columns[1:-1])
test_sentences = list(test_df.sentence.values)
print(test_label_cols)
print('Null values: ', test_df.isnull().values.any()) #should not be any null sentences or labels
print('Same columns between train and test: ', label_cols == test_label_cols) #columns should be the same

['physical', 'mutual', 'self-volitional', 'volitional', 'instrument', 'autonomous', 'cause', 'manipulate', 'internal', 'force', 'provide', 'form', 'place', 'motion', 'dynamictexture', 'remove', 'create', 'location', 'deprive', 'cos', 'attend', 'xpr', 'purp', 'affect', 'none']
Null values:  True
Same columns between train and test:  True


In [48]:
test_df['one_hot_labels'] = list(test_df[label_cols].values)
test_df['one_hot_labels_fd1'] = list(test_df[all_fd1].values)
test_df['one_hot_labels_fd2'] = list(test_df[all_fd2].values)
test_df['one_hot_labels_fd3'] = list(test_df[all_fd3 + ['none']].values)
test_df.head()

,sentence,physical,mutual,self-volitional,volitional,instrument,autonomous,cause,manipulate,internal,...,attend,xpr,purp,affect,none,experience,one_hot_labels,one_hot_labels_fd1,one_hot_labels_fd2,one_hot_labels_fd3
0,Plants die,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,NaN,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 1]"
1,They are buried in sediment,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,NaN,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 1]"
2,Bacteria is buried in sediment,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,NaN,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 1]"
3,Large amounts of sediment gradually pile on to...,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,NaN,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 1, 0, 0, 1]"
4,Pressure builds up,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,NaN,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 1]"


In [49]:
len(test_df.columns)

31

In [50]:
# Gathering input data
test_labels = list(test_df.one_hot_labels.values)
test_sentences = list(test_df.sentence.values)

In [51]:
# Encoding input data
test_encodings = tokenizer.batch_encode_plus(test_sentences,
#                                              max_length=max_length,
                                             padding=True,
                                             return_token_type_ids=True)
test_input_ids = test_encodings['input_ids']
test_token_type_ids = test_encodings['token_type_ids']
test_attention_masks = test_encodings['attention_mask']

In [52]:
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
test_token_types = torch.tensor(test_token_type_ids)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
# torch.save(test_dataloader,'test_data_loader')

In [53]:
# Test

# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels, b_token_types = batch
  with torch.no_grad():
    # Forward pass
    outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    b_logit_pred = outs[0]
    pred_label = torch.sigmoid(b_logit_pred)

    b_logit_pred = b_logit_pred.detach().cpu().numpy()
    pred_label = pred_label.to('cpu').numpy()
    b_labels = b_labels.to('cpu').numpy()

  tokenized_texts.append(b_input_ids)
  logit_preds.append(b_logit_pred)
  true_labels.append(b_labels)
  pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

In [54]:
len(test_label_cols)

25

In [55]:
test_label_cols

['physical',
 'mutual',
 'self-volitional',
 'volitional',
 'instrument',
 'autonomous',
 'cause',
 'manipulate',
 'internal',
 'force',
 'provide',
 'form',
 'place',
 'motion',
 'dynamictexture',
 'remove',
 'create',
 'location',
 'deprive',
 'cos',
 'attend',
 'xpr',
 'purp',
 'affect',
 'none']

In [56]:
pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding

# Print and save classification report
print('Test F1 Accuracy: ', f1_score(true_bools, pred_bools,average='micro'))
print('Test Flat Accuracy: ', accuracy_score(true_bools, pred_bools),'\n')
clf_report = classification_report(true_bools,pred_bools,labels=range(25),target_names=test_label_cols)
#pickle.dump(clf_report, open('../reports/classification_report.txt','wb')) #save report
print(clf_report)

Test F1 Accuracy:  0.7376035775351834
Test Flat Accuracy:  0.2884322678843227 

                 precision    recall  f1-score   support

       physical       0.82      0.27      0.41       297
         mutual       0.00      0.00      0.00         1
self-volitional       0.21      0.25      0.23        12
     volitional       0.46      0.75      0.57       115
     instrument       0.40      0.25      0.31         8
     autonomous       0.85      0.88      0.87       851
          cause       0.18      0.43      0.25        30
     manipulate       0.00      0.00      0.00         0
       internal       0.57      0.34      0.43       186
          force       0.93      0.36      0.52        36
        provide       0.45      0.29      0.36        17
           form       0.00      0.00      0.00       113
          place       0.35      0.69      0.46       124
         motion       0.85      0.25      0.39       260
 dynamictexture       0.00      0.00      0.00         0
       

/nfs/research/regan/.conda/envs/torch-roberta/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/nfs/research/regan/.conda/envs/torch-roberta/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
baseline_labels = ['volitional', 'cos', 'none']

test_label_baseline = [[True if l in baseline_labels else False for l in test_label_cols]]*len(pred_bools)

test_label_baseline[0]

[False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True]

In [66]:
# Baseline majority
print('Test F1 Accuracy: ', f1_score(true_bools, test_label_baseline,average='micro'))
print('Test Flat Accuracy: ', accuracy_score(true_bools, test_label_baseline),'\n')
clf_report_baseline = classification_report(true_bools,test_label_baseline,labels=range(25),target_names=test_label_cols)
print(clf_report_baseline)

Test F1 Accuracy:  0.43987301587301586
Test Flat Accuracy:  0.0121765601217656 

                 precision    recall  f1-score   support

       physical       0.00      0.00      0.00       297
         mutual       0.00      0.00      0.00         1
self-volitional       0.00      0.00      0.00        12
     volitional       0.09      1.00      0.16       115
     instrument       0.00      0.00      0.00         8
     autonomous       0.00      0.00      0.00       851
          cause       0.00      0.00      0.00        30
     manipulate       0.00      0.00      0.00         0
       internal       0.00      0.00      0.00       186
          force       0.00      0.00      0.00        36
        provide       0.00      0.00      0.00        17
           form       0.00      0.00      0.00       113
          place       0.00      0.00      0.00       124
         motion       0.00      0.00      0.00       260
 dynamictexture       0.00      0.00      0.00         0
      

In [57]:
from itertools import compress

def get_image_schema(arr, fil):
    return list(compress(arr, fil))

In [58]:
label_cols

['physical',
 'mutual',
 'self-volitional',
 'volitional',
 'instrument',
 'autonomous',
 'cause',
 'manipulate',
 'internal',
 'force',
 'provide',
 'form',
 'place',
 'motion',
 'dynamictexture',
 'remove',
 'create',
 'location',
 'deprive',
 'cos',
 'attend',
 'xpr',
 'purp',
 'affect',
 'none']

In [59]:
[(sent, true, get_image_schema(label_cols,pred)) for (sent, true, pred) in zip(test_sentences[-16::], true_labels[-16::], pred_bools[-16::])]

[('magma turns into lava in air and travels down volcano',
  array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1]),
  ['autonomous', 'motion', 'none']),
 ('Evaporation of water at surface turns into water vapors',
  array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1]),
  ['autonomous', 'cos', 'none']),
 ('water turns into steam',
  array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1]),
  ['autonomous', 'cos', 'none']),
 ('Turn on washing machine',
  array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 1]),
  ['volitional', 'internal', 'none']),
 ('resulting material disappears',
  array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 1]),
  ['autonomous', 'cos', 'none']),
 ('mud around skeleton turns into rock',
  array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1]),

In [60]:
all_preds = [(sent, get_image_schema(label_cols,true.astype(bool)), get_image_schema(label_cols,pred)) if any(true-pred.astype(int) != 0) else "good" for (sent, true, pred) in zip(test_sentences, true_labels, pred_bools)]

In [61]:
for p in all_preds:
    if p != 'good':
        print(p[0])
        print("Actual:", p[1])
        print("Pred:", p[2])
        print()

Pressure builds up
Actual: ['autonomous', 'cos', 'none']
Pred: ['internal', 'none']

Heat increases
Actual: ['autonomous', 'cos', 'none']
Pred: ['autonomous', 'internal', 'none']

chemical structure of buried sediment and plants changes
Actual: ['autonomous', 'cos', 'none']
Pred: ['autonomous', 'internal', 'none']

dead algae and plankton end up part of sediment on seafloor
Actual: ['autonomous', 'cos', 'none']
Pred: ['autonomous', 'none']

bottom layers of sediment become compacted by pressure
Actual: ['physical', 'cos', 'none']
Pred: ['cause', 'cos', 'none']

Higher pressure causes sediment to heat up
Actual: ['physical', 'cos', 'none']
Pred: ['cause', 'cos', 'none']

heat causes chemical processes
Actual: ['physical', 'internal', 'none']
Pred: ['cause', 'create', 'none']

material becomes liquid
Actual: ['autonomous', 'form', 'none']
Pred: ['autonomous', 'cos', 'none']

Magma rises from deep in earth
Actual: ['autonomous', 'motion', 'none']
Pred: ['autonomous', 'remove', 'none']

ma

Actual: ['autonomous', 'internal', 'none']
Pred: ['autonomous', 'cos', 'none']

Plants start growing on or near rocks
Actual: ['autonomous', 'cos', 'none']
Pred: ['autonomous', 'location', 'none']

Rain falls on wood
Actual: ['autonomous', 'motion', 'none']
Pred: ['autonomous', 'place', 'none']

rain seeps into wood surface
Actual: ['autonomous', 'motion', 'none']
Pred: ['autonomous', 'place', 'none']

It permeates wood
Actual: ['autonomous', 'motion', 'none']
Pred: ['autonomous', 'place', 'none']

When rain evaporates it leaves wood
Actual: ['autonomous', 'motion', 'none']
Pred: ['autonomous', 'cos', 'none']

It takes finish of wood with it
Actual: ['physical', 'remove', 'none']
Pred: ['physical', 'instrument', 'none']

It begins to look bad
Actual: ['autonomous', 'cos', 'none']
Pred: ['autonomous', 'internal', 'none']

Water seeps into rocks
Actual: ['autonomous', 'motion', 'none']
Pred: ['autonomous', 'place', 'none']

Cracks form in rocks
Actual: ['autonomous', 'cos', 'none']
Pred:

Actual: ['physical', 'cos', 'none']
Pred: ['volitional', 'cos', 'none']

truck drives to landfill
Actual: ['physical', 'motion', 'none']
Pred: ['autonomous', 'motion', 'none']

trashbags are thrown into large outdoor trashcan
Actual: ['autonomous', 'place', 'none']
Pred: ['place', 'none']

trash truck travels to landfill
Actual: ['physical', 'motion', 'none']
Pred: ['autonomous', 'motion', 'none']

Unloads all trash truck has collected
Actual: ['autonomous', 'remove', 'none']
Pred: ['volitional', 'internal', 'none']

trash is picked up by workers who put it on garbage truck
Actual: ['volitional', 'remove', 'none']
Pred: ['volitional', 'place', 'none']

garbage is sorted
Actual: ['autonomous', 'cos', 'none']
Pred: ['volitional', 'cos', 'none']

Some of garbage is saved for recycling
Actual: ['autonomous', 'internal', 'purp']
Pred: ['autonomous', 'cos', 'purp']

Cover up hole
Actual: ['volitional', 'provide', 'none']
Pred: ['volitional', 'cos', 'none']

You wait for seeds to sprout
Actua

Actual: ['physical', 'form', 'none']
Pred: ['physical', 'cause', 'cos', 'none']

